# 机械式激光雷达的无标定物标定算法

## 一、机器人手眼标定
现在的机器人少不了有各种传感器，传感器之间的标定是机器人感知环境的一个重要前提。手眼标定的核心公式只有一个
$$AX=XB$$
这里的$X$就是指手（机械臂末端）与眼（摄像头）之间的坐标转换关系。下面结合机械臂的两种使用场景，讲一下这个公式的由来。

### 1.1 Eye-in-Hand
所谓Eye-In-Hand，是指摄像头被安装在机械臂上。此时要求End到Camera之间的坐标转换关系，即
$$X=T_{E\rightarrow C}$$
<p align="center"><img src="../resources/spin-target/eye-in-hand.jpg" width=50%></p>

**注意：**$T_{A\rightarrow B}$即传感器B相对于传感器A的外参，$X_A = T_{A\rightarrow B}X_B$将传感器B下的坐标转换到传感器A下坐标。

这种情况下，有两个变量是不变的：
* 摄像头与机械臂末端之间的坐标转换关系不变，也就是说，$T_{E\rightarrow C}$始终不变；
* 标定板与机械臂底座之间的坐标转换关系不变，也就是说，$T_{B\rightarrow O}$ 也是不变的。

将前后两帧的$T_{B\rightarrow O}$展开，得到
$$T_{B\rightarrow O} = T_{B\rightarrow E_1} * T_{E_1\rightarrow C_1} * T_{C_1\rightarrow O} = T_{B\rightarrow E_2} * T_{E_2\rightarrow C_2} * T_{C_2\rightarrow O} $$
$$T_{E_2\rightarrow E_1} * T_{E_1\rightarrow C_1} =  T_{E_2\rightarrow C_2} * T_{C_2\rightarrow C_1} $$
令$A:= T_{E_2\rightarrow E_1} \quad X:= T_{E\rightarrow C} \quad B:= T_{C_2\rightarrow C_1}$
$A, B$已知，$X$未知，即转为求解$AX=XB$线性方程组。

### 1.2 Eye-to-Hand
所谓Eye-To-Hand，是指摄像头被安装在一个固定不动的位置，而标定板被拿在机械臂手上。此时要求取的是，Base到Camera之间的坐标转换关系，也就是 $T_{B\rightarrow C}$。这种情况下，有两个变量是不变的：
<p align="center"><img src="../resources/spin-target/eye-to-hand.jpg" width=50%></p>

* 摄像头与机械臂底座之间的坐标转换关系不变，也就是说， $T_{B\rightarrow C}$始终不变；
* 标定板与机械臂末端之间的坐标转换关系不变，也就是说， $T_{E\rightarrow O}$始终不变。
把$T_{E\rightarrow O}$展开
$$T_{E\rightarrow O} = T_{E_1\rightarrow B} * T_{B\rightarrow C} * T_{C\rightarrow O_1} = T_{E_2\rightarrow B} * T_{B\rightarrow C} * T_{C\rightarrow O_2} $$
令$A:= (T_{E_2\rightarrow B})^{-1} * T_{E_1\rightarrow B} \quad X:= T_{B\rightarrow C} \quad B:= T_{C\rightarrow O_2}* (T_{C\rightarrow O_1})^{-1}$
$A, B$已知，$X$未知，即转为求解$AX=XB$线性方程组。

### 1.3 求解方法
见参考资料[5]


## 一、文献[3]
文献[3]研究了机械式激光雷达与可见光相机之间的标定问题，同样采用**边缘信息**进行标定。

### 1.1 图像预处理
对图像的处理过程是将图像会度化，再用边缘提取算子提取边缘获得边缘图E。接着用逆距离变换(inverset distance transform)对边缘图E进行平滑变换成一张强度衡量"edgness"边缘程度的图像D。

$$ D_{i,j} = \alpha E_{i, j} + (1 - \alpha) \cdot \max_{x, y}E_{x, y} \cdot \gamma^{\max(|x - i|, |y - j|)} \quad (\alpha = \frac{1}{3}, \gamma = 0.98)$$ 

<p align="center"><img src="../resources/spin-target/paper1_ip.png" width=50%></p>

### 1.2 激光雷达提取边缘
文献[3]提取的边缘属于不连续的边缘，即对相邻扫描点进行差分处理，然后将距离小于0.3m的点滤除，即获得不连续边缘点。

$$X_P^i = \max(P_{p-1}^i\cdot r - P_{p}^i\cdot r, P_{p + 1}^i\cdot r) - P_{p}^i\cdot r, 0$$

### 1.3 目标函数
就是将三维点云投影到图像D上，对投影点的邻域“edgeness”进行加权求和(类似Spatial Transformer Network)，接着使用LM算法对$R, t$进行迭代优化。

$$\max_{R, t}J_{R, t} = \sum_i C(\Pi_{R, t}(X_P^i), D)$$

**注意**:这和边到边的ICP算法有本质不同，边到边的ICP算法每一次迭代前会计算最近边缘，动态构造损失函数。


## 二、文献[4]
文献[4]是基于手眼标定的标定方法，即$AX=XB$。
<p align="center"><img src="../resources/spin-target/paper2_method.png" width=30%></p>

配准流程如下

<p align="center"><img src="../resources/spin-target/paper2_stream.png" width=50%></p>



## 参考资料
[1] [机器人手眼标定](https://zhuanlan.zhihu.com/p/103749589)

[2] [知乎-多传感器标定](https://zhuanlan.zhihu.com/p/57028341)

[3] [Automatic Online Calibration of Cameras and Lasers](http://roboticsproceedings.org/rss09/p29.pdf)

[4] Ishikawa, R., Oishi, T., & Ikeuchi, K. (2018). LiDAR and Camera Calibration using Motion Estimated by Sensor Fusion Odometry. ArXiv Preprint ArXiv:1804.05178.

[5] [知乎-手眼标定求解方法](https://zhuanlan.zhihu.com/p/78855930)